# Billboard exercise

In [1]:
import numpy as np
import cv2

Load the two images:

In [2]:
base_img = cv2.imread("samples/billboard.jpg")
base_img = cv2.resize(base_img, dsize=[1920, 1080], dst=None)
img_copy = base_img.copy()
img2 = cv2.imread("samples/landscape.jpeg")

Now, we can get the image data:

In [3]:
base_h, base_w = base_img.shape[:2]
img2_h, img2_w = img2.shape[:2]

We can create source and destination points.

In [4]:
src_points = np.array([
    [0, 0],
    [0, img2_h],
    [img2_w, img2_h],
    [img2_w, 0]
], dtype=np.float32)

dst_points = []     #this will be filled with the clicks of the mouse

We have to define the window for getting the points:

In [5]:
def onClick(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(dst_points) < 4:
            dst_points.append([x, y])
            cv2.circle(img_copy, (x, y), 50, (0, 0, 255), -1)
            cv2.imshow("Base image", img_copy)

In [6]:
cv2.namedWindow("Base image", cv2.WINDOW_KEEPRATIO)
cv2.setMouseCallback("Base image", onClick)

Show the image in order to be clicked:

In [7]:
cv2.imshow("Base image", base_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Get the homography matrix:

In [8]:
dst_float = np.array(dst_points, dtype = np.float32)    #the transformation can only happen with floats!
H = cv2.getPerspectiveTransform(src_points, dst_float)

Apply the homography matrix to the image angles:

In [9]:
warped = cv2.warpPerspective(img2, H, (base_w, base_h))
cv2.imshow("Warped image", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

A mask can be employed to drop black pixels from the warped image. This is because the warped image contains black in all its parts that are not occupied by the warped one.

In [10]:
mask = np.zeros(base_img.shape, dtype=np.uint8)
cv2.fillConvexPoly(mask, np.int32(dst_float), (255, 255, 255))    #this is black except for the warped portion
mask = cv2.bitwise_not(mask)
masked_bill = cv2.bitwise_and(base_img, mask)           #this makes the billboard black
final_img = cv2.bitwise_or(masked_bill, warped)         #ORing the billboard and the warped image

In [11]:
cv2.namedWindow("Final image", cv2.WINDOW_KEEPRATIO)
cv2.imshow("Final image", final_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

True